<div style="border:solid green 2px; padding: 20px">
<font color=green><b>Комментарий ревьюера<b></font>

<b>Привет!</b> Спасибо тебе за выполненный проект:)  
Меня зовут Лена Сидорова и я буду проверять его.
Предлагаю общаться на "ты", но если по какой-то причине тебе так некомфортно, можно и на "вы". 


По ходу тетрадки ты найдешь комментарии к проекту, они будут в отдельных ячейках Markdown с заголовком «Комментарий ревьюера».
Постарайся учесть эти комментарии для дальнейших проектов.

Пожалуйста, не перемещай, не изменяй и не удаляй мои комментарии. Всё это поможет выполнить повторную проверку твоего проекта оперативнее.

---

Комментарии будут выделены цветом:

<div class="alert alert-block alert-danger">
    

 __Комментарий от ревьюера__

Такой комментарий нужно исправить обязательно, он критически влияет на удачное выполнение проекта.
</div>
    
---

<div class="alert alert-block alert-warning">
    
__Комментарий от ревьюера__


Такой комментарий является рекомендацией или советом. Можешь использовать их на своё усмотрение.
</div>

---

<div class="alert alert-block alert-success">
    

__Комментарий от ревьюера__

Такой комментарий  говорит о том, что ты замечательно справился с задачей :)
</div>

---


Предлагаю работать в диалоге, если что-то меняешь по моим рекомендациям, пиши об этом. Выбери свой любимый цвет, чтобы твои комментарии были заметнее)   
Также можешь использовать такую конструкцию, чтобы оформить свой ответ:  


---

<div class="alert alert-block alert-info">
    

__Ответ__

Здесь можно что-то написать
</div>

---

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd

df = pd.read_csv('/datasets/data.csv')
df.info()  
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

У некоторых не заполнен стаж и не заполнен доход, более того у одних и тех же. Возможно не захотели заполнять, возможно данные неизвестны, возможно проблема в другом.

Стаж имеет тип float, что излишне, int достаточно. Для уровня дохода - аналогично. У стажа бОльшая часть данных - отрицательные числа. Возможно это проблема занесения данных. 

id образования проставлено нормально, то есть каждому id соответствуют одинаковые по смыслу строки образования. Семейный статус - аналогично.

Отрицательное количество детей. 

<div class="alert alert-block alert-success">
    

__Комментарий от ревьюера__

Отлично, фронт работы намечен
</div>

<div class="alert alert-block alert-warning">
    

__Комментарий от ревьюера__

Также тут хорошо бы вывести первые 5-10 строк датасета для ознакомления и описание данных, что означает каждая колонка в датасете
</div>

<div class="alert alert-block alert-info">
    Исправил
</div>

<div class="alert alert-block alert-success">
    

__Комментарий от ревьюера v2__

👌
</div>

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:

f = lambda x: max(x,-x)
print(df.isna().sum())
#Исправление отрицательных чисел
df['children'] = df['children'].apply(f)
df['days_employed'] = df['days_employed'].apply(f)

#Часть стажа занесена неадекватно. Возможна ситуация, когда стаж больше количества лет клиента
#Для определения такаих записей делю количество дней на 365, получается примерно лет стажа. 
#Заменю на 0 что бы данные значения не смазывали статистику. При обработке стажа 
#такие данные учитывать нельзя.
df['days_employed'] = df.apply(lambda x: 0 if (x['days_employed']/365>x['dob_years']) else x['days_employed'], axis = 1)

print('Неорректных записей пола ' + str(df[df['gender']=='XNA']['gender'].count()))
df.loc[df['gender']=='XNA','gender'] = 'M'

#Медианное значение для дохода и стажа буду заполнять в разрезе занятости и пола
meredian_days = df[df['days_employed']>0].groupby(['income_type','gender'])['days_employed'].median().reset_index()
meredian_income = df[df['total_income']>0].groupby(['income_type','gender'])['total_income'].median().reset_index()
 
for i in meredian_days.iterrows():
    mask = (df['income_type'] ==i[1]['income_type'])&(df['gender'] ==i[1]['gender']) &(df['days_employed'].isnull())
    df.loc[mask,'days_employed'] = i[1]['days_employed']
for i in meredian_income.iterrows():
    mask = (df['income_type'] ==i[1]['income_type'])&(df['gender'] ==i[1]['gender']) &(df['total_income'].isnull())
    df.loc[mask,'total_income'] = i[1]['total_income']
    
#Остатки попробую заполнить только в разрезе занятости    
meredian_days = df[df['days_employed']>0].groupby('income_type')['days_employed'].median().reset_index()
meredian_income = df[df['total_income']>0].groupby('income_type')['total_income'].median().reset_index()

for i in meredian_days.iterrows():
    mask = (df['income_type'] ==i[1]['income_type']) &(df['days_employed'].isnull())
    df.loc[mask,'days_employed'] = i[1]['days_employed']
for i in meredian_income.iterrows():
    mask = (df['income_type'] ==i[1]['income_type']) &(df['total_income'].isnull())
    df.loc[mask,'total_income'] = i[1]['total_income']
    
#Если не исправились, дорисую нули
df['days_employed'] = df['days_employed'].fillna(0)
df['total_income'] = df['total_income'].fillna(0)


print(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
Неорректных записей пола 1
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


**Вывод**

Для исправления стажа привожу кго к положительному числу. Есть записи где стаж больше возраста клиента, такие обнуляю, потому что такие записи у всех пенсионеров и у всех безработных. По таким данным нельзя вычислить корректную медианную в рамках занятости и заполнить, а также сильно размывает средние значения в целом. Если обнулю, то легко фильтровать такие записи и средний стаж в целом будет более корректный. Количество дней стажа делю на 365, получаю число лет стажа и сравниваю с возрастом.

Указан непонятной пол XNA. Он всего один, можно заменить на М, на статистику не повлияет.

Потом nan заполняю медианным значением в разрезе типа занятости и пола. Если остаются nan, то только по занятости. Остальные nan обнуляю.

<div class="alert alert-block alert-danger">
    

__Комментарий от ревьюера__

В проектах ячейки с кодом следует дополнять комментариями, что ты в них делаешь. Так твой проект будет понятен читателям, даже те кто не разбираются в коде. А зачастую такие бывают.   
Прокомментрируй, пожалуйста, свои решения. Если с положительными числами все более менее понятно, то с заполнением пропусков медианной не очень.  Тут стоит ответить на следующие вопросы:  
    
    
1. Почему ты решил заполнить именно медианной?
2. Как ты считаешь, откуда появились такие пропуски?
3. Что означает конструкция `df['days_employed']/365`, для чего ты делишь на 365? 
4. И наконец, последний момент, если все же заполняем медианной, то не совсем будет правильным не учитывать какой тип занятости у человека. Согласись, было бы странно пенсионеру ставить доход такой же как и предпринимателю. Поэтому давай заполним пропуски в соответствии с типом занятости. Возможно у тебя будут еще какие-то идеи. Можно также учесть возрастную группу. В общем, тут есть еще над чем подумать для наиболее достоверного заполнения пропусков.
</div>

<div class="alert alert-block alert-info">  
1. Nan записей достаточно много, если их просто обнулять, то их нужно потом не учитывать в итогах, что бы не поплыла  статистика. Мне проще медианными заполнить.    
2. Данные не были заполнены. Неизвестно как они собирались. Возможно в анкетах люди не указывали.    
3,4 - дозаполнил в выводе
</div>

<div class="alert alert-block alert-success">
    

__Комментарий от ревьюера v2__

Хорошая идея учесть еще и пол клиенты, по остальным пунктам стало тоже гораздо понятнее, согласна на счет медианны
</div>

### Замена типа данных

In [3]:
df.loc[df['children']==20,'children'] = 2

df['total_income']= df['total_income'].astype('int')
df['days_employed']= df['days_employed'].astype('int')




**Вывод**

20 детей - подозрительно много, тем более в 76 записях. Скорее всего это 2 ребёнка, 0 могли случайно задеть при занесении на доп цифровой клавиатуре. Их нужно либо не учитывать, либо отнести на что-то. Отнесу к 2 детям, так как это вероятно правильно и не изменит глобально статистику.

Преобразовал тип из float в int


<div class="alert alert-block alert-success">
    

__Комментарий от ревьюера__

Да согласна. Только вот, где мы выяснили что пол XNA всего один? Также стоит добавить, что ты преобразовал типы данных
</div>

### Обработка дубликатов

In [4]:
df = df.drop_duplicates().reset_index(drop=True)

**Вывод**

Дубликаты обнуляю только полные. Так как я не знаю за какой период эта таблица сделана и нет ID клиента. В таком случае, клиенты с одинаковым количеством детей, з/п, образованием, стажем, даже с похожими целями, могут быть людьми,  приходившие в разные время. Даже полные дубли могут быть так истолкованы, но вероятность этого минимальная, да и количество таких дублей небольшое, поэтому удаление особо не повлияет.

<div class="alert alert-block alert-success">
    

__Комментарий от ревьюера__

Хорошее решение, соглашусь
    
    
</div>

<div class="alert alert-block alert-danger">
    

__Комментарий от ревьюера__

А как ты считаешь, вообще какие возможные причины появления дубликатов ?
    
    
</div>

<div class="alert alert-block alert-info">  
Возможно кто-то ходил по нескольким филиалам, и разные филиалы слили одного и того же человека в общую базу. Возможно проблема в СУБД, кто-то неправильно пользуется левым соединением. Возможно поменялись какие-то критические данные у клиента, типа телефона, паспорта, его занесли второй раз. Возможно ошибка оператора.
</div>

<div class="alert alert-block alert-success">
    

__Комментарий от ревьюера v2__

Да, вполне возможные причины. 
</div>

### Лемматизация

In [5]:
from pymystem3 import Mystem
m = Mystem()

def func(x, mas):
    return list(set(x).difference(set(mas)))

#Настройки категоризации. Ключ справочника - категория, значение - массив.
#Если в леммах найдено хотя бы одно значение, то ключ является категорией
#Если элемент настройки массив, значит все слова должны быть в лемме.
cat_purpose = {'ремонт жилья':[['ремонт','жилье']],'автомобиль':['автомобиль']}
cat_purpose['свадьба'] = ['свадьба']
cat_purpose['строительство'] = ['строительство']
cat_purpose['образование'] = ['образование']
cat_purpose['операции с недвижимостью'] = ['жилье','недвижимость']


def purpose_list_group(row):
    purpose_list = m.lemmatize(row['purpose'])   
    for i in cat_purpose:
        #хотя бы одно слово должно находится в целях
        for pur in cat_purpose[i]:
            if type(pur)==type([]):
                #все слова в массива должны быть в целях
                if set(pur) <= set(purpose_list):
                    return i 
            else:
                if pur in purpose_list:
                    return i
                
df['purpose'] = df['purpose'].str.lower()
df['purpose_list_group'] = df.apply(purpose_list_group, axis=1)
print(df["purpose"].unique())
print(df["purpose_list_group"].unique())

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']
['операции с недвижимостью' '

**Вывод**

Лемматизировал цели кредита. Выделил смысловые категории в целях. Распределил цели по категориям. 

По целям кредита - К примеру,есть лемма "операция". Из просто из подсчёта не ясно это медицинская операция или операция с чем-то. Понятно что в данном случае это операция с недвижимостью, но вообще и в принципе такое лучше рассматривать в контексте. Написал совсем простой механизм, думаю достаточно этого.


<div class="alert alert-block alert-warning">
    

__Комментарий от ревьюера__

Отлично, но только лишь это ты сделал? Прокомментируй, пожалуйста, для чего ты применяешь Counter и lemmatize? Что в результате мы хотим получить?
    
    
</div>

<div class="alert alert-block alert-info">  
Подправил. Перенёс сюда категоризацию по леммам. Подсчёт был просто что бы показать информацию о самых частых словах. Смысловой нагрузки было минимально, убрал.
</div>

### Категоризация данных

In [6]:
def children_group(row):
    children = row['children']    
    if children == 0:
        return '0'
    elif children == 1:
        return '1'
    elif children == 2:
        return '2'
    else:
        return '>=3'
    
median = df[df['total_income']>0]['total_income'].median() 
min_measure = median*0.80
max_measure = median*1.20
def total_income_group(row):
    total_income = row['total_income']
    cel = int(total_income // max_measure)
    if total_income > 1000000:
        return 'миллионеры'
    elif cel>0:
        return 'очень '*(cel-1)+'выше среднего'
    elif total_income < min_measure:
        return 'ниже среднего'
    else:
        return 'средний'

    
df['children_group'] = df.apply(children_group, axis=1)
df['total_income_group'] = df.apply(total_income_group, axis=1)


**Вывод**

Категоризовать нужно:
по наличию детей - 0, 1, 2, 3 и более

по доходу - медианное значение  +- 20% это средний доход. Очевидно есть ниже среднего, выше среднего


<div class="alert alert-block alert-success">
    

__Комментарий от ревьюера__

Хорошее решения для категоризации по наличию детей и дохода
    
</div>

<div class="alert alert-block alert-danger">
    

__Комментарий от ревьюера__

По поводу целей кредита у меня вопрос,  для чего же мы выше лемматризировали цели? Если в итоге лемматизацию не применили для категоризации.  Для начала прокомментируй, какие леммы мы выявили, и на какие группы ты решил категоризировать в итоге. Это явно не видно.  Далее на данном этапе хорошо бы научиться применять лемматизацию непосредственно для выделения категорий. Я бы хотела увидеть от тебя функцию, которая сразу ищет в леммах основные наши цели и присваивает значения в соответсвии с ними. Примерно это должно выглядеть так:  
    


`def purpose_list_group(row):
    lem_purpose = m.lemmatize(row['purpose'])
    try:
        if 'автомобиль' in lem_purpose:
            return 'автомобиль'
        if ('жилье' in lem_purpose) or ('недвижимость' in lem_purpose ):
            return 'операции с недвижимостью' `
    
    
 Пока эти две части у тебя отделены, ты отдельно ищешь леммы, отдельно в соответсвии со словарем присваиваешь категории. Это в принципе выполняет нашу цель, но делает это не совсем оптимально.
    

    
</div>

<div class="alert alert-block alert-info">  
Подправил. Категоризацию по целям перенёс выше.
</div>

<div class="alert alert-block alert-success">
    

__Комментарий от ревьюера v2__

Супер, код стал компактнее и на мой взгляд более оптимальным
</div>

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:

table = pd.pivot_table(df, index=['children_group'], values=['children','debt'],
           aggfunc={'children':'count','debt': 'sum'})
table['children'] = table['children']/len(df)*100
table['debt'] = table['debt']/df['debt'].sum()*100
table['koef'] = table['debt']/table['children']
print(table.sort_values("koef", ascending=False))

                 children       debt      koef
children_group                                
2                9.911043  11.602527  1.170667
1               22.616553  25.560023  1.130147
>=3              1.769829   1.780586  1.006078
0               65.702576  61.056864  0.929292


**Вывод**

Да. Люди без детей гораздо чаще возвращают долг. Люди с детьми - чаще не возвращают. 

Здесь и ниже в сводной таблице данные описаны в следющем формате.

Категория | Доля категории среди всех клиентов | Доля клиентов данной категории с долгом среди всех клиентов с долгом | Коэфициент долга

Коэфициент долга = Доля клиентов данной категории с долгом среди всех клиентов с долгом / Доля категории среди всех клиентов

Данный коэф показывает сколько % должников данной группы приходится на 1% клиентов из данной группы



<div class="alert alert-block alert-success">
    

__Комментарий от ревьюера__

Здорово, что применил для ответа на вопрос сводную таблицу, она отлично здесь подходит.  

    
</div>

<div class="alert alert-block alert-warning">
    

__Комментарий от ревьюера__

Прокомментируй, пожалуйста, как мы приходим к такому выводы,  какие вычисленные столбцы нам в этом помогают и как мы их вычисляем.  
    
Например, нигде не написано, что означает колонка `debt`, что ты вычисляешь в колонке  `children` и `koef`?   
    
Не думай, что я придираюсь, но тут важно понять, что твои вычисления очевидны только для тебя, но для твоей аудитории(те, кто будет читать этот проект), некоторые моменты могут быть непонятны и их нужно комментировать, чтобы твой ход мыслей был виден всем.  К примеру, для меня было бы логичным посмотреть на долю возврата среди общего количества в каждой группе, а ты вычисляешь две других доли и делишь их между собой. Давай обсудим этот момент

    
</div>

<div class="alert alert-block alert-info">  
Какие столбцы - описал в выводе. В таком формате я вижу и сколько % от всех занимает данная группа и какой % от общих долгов у клиентов данной группы. Таким образом я могу фильтровать группы по важности, при необходимости. К примеру клиенты без детей 65.702576 ; 61.056864 ; 0.929292, то есть у 65.7% всех клиентов нет детей, у них 61.05% всех должников. Делю одно на другое, получаю, что на 1% клиентов в данной группе приходится 0.93% должников. Мне кажется, это адекватный показатель, достаточно удобный и даёт право делать выводы. Можно и как описала, результаты буду одинаковы. 
</div>

<div class="alert alert-block alert-success">
    

__Комментарий от ревьюера v2__

Да, это действительно интересный подход. Очень важно прописывать такие комментарии с логикой вычисления, теперь вычисления стали наиболее прозрачными
</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
table = pd.pivot_table(df, index=['family_status_id'], values=['family_status','debt'],
           aggfunc={'family_status':'count','debt': 'sum'})
table['family_status'] = table['family_status']/len(df)*100
table['debt'] = table['debt']/df['debt'].sum()*100
table['koef'] = table['debt']/table['family_status']
print(table.sort_values("koef", ascending=False))

df[['family_status','family_status_id']].groupby(['family_status_id','family_status']).count()

                       debt  family_status      koef
family_status_id                                    
4                 15.738082      13.087420  1.202535
1                 22.286043      19.388943  1.149420
0                 53.475014      57.491500  0.930138
3                  4.882252       5.565647  0.877212
2                  3.618610       4.466490  0.810169


,
family_status_id,family_status
0,женат / замужем
1,гражданский брак
2,вдовец / вдова
3,в разводе
4,Не женат / не замужем


**Вывод**

Да. Не женат/не замужем чаще не возвращают, те кто уже был в браке чаще возвращают

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
table = pd.pivot_table(df, index=['total_income_group'], values=['total_income','debt'],
           aggfunc={'total_income':'count','debt': 'sum'})
table['total_income'] = table['total_income']/len(df)*100
table['debt'] = table['debt']/df['debt'].sum()*100
table['koef'] = table['total_income']/table['debt']
print(table.sort_values("koef", ascending=False))
#уберу записи <1% ,отнесу к статистической погрешности
print(table[table['debt']>1].sort_values("koef", ascending=False))

df['total_income_group'].value_counts()

                                            debt  total_income      koef
total_income_group                                                      
очень очень выше среднего               0.344630      0.596153  1.729837
очень очень очень выше среднего         0.114877      0.144381  1.256835
очень выше среднего                     2.871913      3.362675  1.170884
выше среднего                          27.627800     29.211495  1.057322
миллионеры                              0.114877      0.116436  1.013576
ниже среднего                          29.465824     29.532858  1.002275
средний                                39.402642     36.998742  0.938991
очень очень очень очень выше среднего   0.057438      0.037260  0.648689
                          debt  total_income      koef
total_income_group                                    
очень выше среднего   2.871913      3.362675  1.170884
выше среднего        27.627800     29.211495  1.057322
ниже среднего        29.465824     29.532858  1.00

средний                                  7944
ниже среднего                            6341
выше среднего                            6272
очень выше среднего                       722
очень очень выше среднего                 128
очень очень очень выше среднего            31
миллионеры                                 25
очень очень очень очень выше среднего       8
Name: total_income_group, dtype: int64

**Вывод**

Более менее достоверная выборка для дохода менее чем медианный доход * 1.2 * 3. Можно сделать вывод, что при больших заработках чаще не возвращают. Средний заработок возвращает чаще, но это может быть связано, что заполняли пропуски медианным доходом.

<div class="alert alert-block alert-warning">
    

__Комментарий от ревьюера__

Интересные группы у тебя получились )) Тебе не кажется, что в целом их можно объединить в одну?  
    
    
`очень очень очень очень очень выше среднего   
очень очень очень выше среднего   
очень очень выше среднего   
очень выше среднего   `

    
</div>

<div class="alert alert-block alert-info">  
Не кажется. Доход может расти очень много и очень резко. Получается объединёная группа группа "выше среднего" может содержать как и 2 раза выше среднего так и в 10 раз выше среднего. Логично предположить, что у людей с явно разным доходом поведение может отличаться. Поэтому надо нарезать доход на группы. Использовать какое то абсолютное значение, типа шаг 10 000 - не хочется, поэтому использую нарезку на интервалы относительно медианного дохода. Ключевое, я изначально не знаю сколько в какую группу попадёт. Будет ли достаточная выборка у данного интервала. После вычисления, я могу определить группы с недостаточным количеством участников и могу такие группы выкинуть, но объединить, в общем случае, нет, так как группы могут разительно отличаться по смыслу, наполености и прочему. В данном конкретном случае, может и возможно руками объединить, но смысла я в этом не вижу, так как это менее 1% данные, притом вообще из разных уровней дохода и только смазывает всё.
</div>

<div class="alert alert-block alert-success">
    

__Комментарий от ревьюера v2__

Ок, поняла тебя. Но названия все же тяжелы для восприятия. Возможно, тут подошло бы что-то типа "доход в 3 раза выше среднего".
</div>

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
table = pd.pivot_table(df, index=['purpose_list_group'], values=['purpose','debt'],
           aggfunc={'purpose':'count','debt': 'sum'})
table['purpose'] = table['purpose']/len(df)*100
table['debt'] = table['debt']/df['debt'].sum()*100
table['koef'] = table['purpose']/table['debt']
table.sort_values("koef", ascending=False)


,debt,purpose,koef
purpose_list_group,,,
ремонт жилья,2.010339,2.827069,1.406265
операции с недвижимостью,34.635267,38.787201,1.119876
строительство,8.271109,8.751339,1.058061
свадьба,10.683515,10.875134,1.017936
образование,21.252154,18.694984,0.879675
автомобиль,23.147616,20.064273,0.866796


**Вывод**

Те кто берут кредит на ремонт жилья гораздо чаще не возвращают. Те кто берут на автомобиль и образование, возвращают чаще.

<div class="alert alert-block alert-danger">
    

__Комментарий от ревьюера__

    
А что указано в табличке? Вывод для этого и существует, чтобы описать результаты и выдвинуть свои гипотезы/размышления/предположения/рекомендации

    
</div>

<div class="alert alert-block alert-danger">
    

__Комментарий от ревьюера__

Общий комментарий по этому пункту :   
    
    Здорово, что применил сводные таблицы, но сам порядок вычисления коэффициента остался не очень понятным. Поясни, его пожалуйста, чтобы я смогла понять, почему ты его решил вычислить таким образом. От этого будут и зависеть и ответы на вопросы
    
</div>

<div class="alert alert-block alert-info">  
Подправил.
</div>

<div class="alert alert-block alert-success">
    

__Комментарий от ревьюера v2__

👌
</div>

## Шаг 4. Общий вывод

Исходные данные были не совсем корректны.

Количество лет стажа у безработных и пенсионеров превышало возраст клиентов. Исходя из этого адекватный анализ по стажу для данных групп не предоставляется возможным. Пустные значения стажа совпадает с пустым значением дохода, что говорит о возможной ошибке формирования таблицы, либо о неполных начальных данных. Очень много отрицательного стажа. 

Пустые данные были заполнены медиаными значениями в разрезе типа занятости и пола. 

Произведена категоризация по целям кредита. Были выбраны наиболее близкие и достаточно общие по смыслу категории.

Во всех представленных случаях зависимость есть. В случаях с доходом - достаточно слабая до определённого уровня дохода, потом возрастает. Возможно обеспеченные люди хуже возвращают кредит, но на текущих данных проверить гипотезу невозможно. 


<div class="alert alert-block alert-danger">
    

__Комментарий от ревьюера__

Это твой первый серьезный проект, и я понимаю, что некоторые моменты вызывают вопросы. Но общий вывод по проекту это очень важная его часть. Ты проделал большую работу : выявил пропуски и дубликаты, предобработал данные, категоризировал их, удалил дубликаты и выявил различные закономерности в ходе ответов на вопросы. Опиши тезисно, что ты сделал в проекте. Возможно, стоит дать рекомендации коллегом по выгрузке данных, предположить откуда у нас появились пропуски. Ведь тема проекта : предобработка данных, а в выводе об этом не сказано ни слова.
    
</div>

<div class="alert alert-block alert-info">
    Исправил
</div>

<div class="alert alert-block alert-success">
    

__Комментарий от ревьюера v2__

Здорово, общий вывод стал намного лучше :)
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.

<div style="border:solid green 2px; padding: 20px">
<font color=green><b>Общий комментарий ревьюера<b></font>

Алексей, ты проделал огромную работу, молодец. В целом ты неплохо справился. Ты умеешь предобрабатывать данных, лемматизировать их, строить сводные таблицы. Но проект нуждается в доработке:  
    
 ~~1) В начале проекта стоит добавить описание данных и вывести первые 5-10 строк датасета; ~~  
 ~~2) По ходу работу, твои шаги стоит комментировать и пояснять свои решения; ~~  
 ~~3) При заполнение пропусков, нужно учесть их тип занятости (возможно, у тебя появятся еще какие-то идеи), а не просто заполнять их медианной; ~~  
 ~~4) Стоит назвать возможные причины возникновения дубликатов; ~~  
 ~~5) При лемматизации целей кредита, нужно применить ее для выявления новый категорий с помощью функции. Если здесь у тебя возникнут трудности, постараемся разобраться с ними вместе. Цель  - научиться применять лемматизацию непосредственно для выделения категорий; ~~    
 ~~6) При ответах на вопросы - поясни, пожалуйста, что за коэффициенты ты вычислял и почему именно так; ~~    
 ~~7) Из таблиц не всегда очевидно понятен вывод, вывод для того и существует, чтобы описать полученные результаты; ~~    
 ~~8) Общий вывод проекта - это почти центральная его часть, многие заказчики первым делом посмотрят на него.И лишь потом примут решения изучать ли подробно твой проект или достаточно все описано в выводе.  Поэтому стоит его дополнить. ~~     
    
    
 Жду твой доработанный проект :)
    
    
<font color=green><b>Общий комментарий ревьюера v2<b></font>  
    
Проект очень качественно доработан, по всем пунктам сделаны значимые дополнения, поздравляю! Проект принят :)  
Желаю тебе успехов! 🥳
